# Analysis for ABC Telecom companey to decide its marketing stretegy in brussels

In [1]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.4.1-py_0        conda-forge
    folium:          

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate


import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocod
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Get all Brussels municipalities 

In [30]:
# Get data from wikipedia
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_municipalities_of_the_Brussels-Capital_Region').text
soup = BeautifulSoup(website_text,'lxml')

# Look for the table in the source html with class wikitable sortable and read all the rows in a list 
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')


data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [31]:
df_raw = pd.DataFrame(data, columns=['SR','FrenchName', 'DutchName','a','b','Postalcode', 'Population','area','density'])

In [32]:
# Get names of indexes for which column Age has value 30
indexNames = df_raw[ df_raw['DutchName'] == 'None' ].index
# Delete these row indexes from dataFrame
df_raw.drop(indexNames , inplace=True)
#indexNames

In [33]:
df_raw = df_raw[~df_raw['DutchName'].isnull()]  
df_raw.head()

,SR,FrenchName,DutchName,a,b,Postalcode,Population,area,density
1,1,Anderlecht,Anderlecht,,,1070,"118,241",17.717.7 km2 (6.8 sq mi),"6,680"
2,2,Auderghem,Oudergem,,,1160,"33,313",09.09.0 km2 (3.5 sq mi),"3,701"
3,3,Berchem-Sainte-Agathe,Sint-Agatha-Berchem,,,1082,"24,701",02.92.9 km2 (1.1 sq mi),"8,518"
4,4,Bruxelles-Ville*,Stad Brussel*,,,1000102011201130,"176,545",32.632.6 km2 (12.6 sq mi),"5,415"
5,5,Etterbeek,Etterbeek,,,1040,"47,414",03.13.1 km2 (1.2 sq mi),"15,295"


## Get the location cordinates for given municipalities

In [34]:
def return_latlng(address):
    getCodeurl = 'http://www.mapquestapi.com/geocoding/v1/address?key=zYeCyag93JyhhM6xofvLSGjc8HKKYxB1&location='+address+',Belgium'
    return requests.get(getCodeurl).json()['results'][0]['locations'][0]['latLng']

In [35]:
df_final = df_raw.drop(['a','b'],1)

In [36]:
df_raw.head()

,SR,FrenchName,DutchName,a,b,Postalcode,Population,area,density
1,1,Anderlecht,Anderlecht,,,1070,"118,241",17.717.7 km2 (6.8 sq mi),"6,680"
2,2,Auderghem,Oudergem,,,1160,"33,313",09.09.0 km2 (3.5 sq mi),"3,701"
3,3,Berchem-Sainte-Agathe,Sint-Agatha-Berchem,,,1082,"24,701",02.92.9 km2 (1.1 sq mi),"8,518"
4,4,Bruxelles-Ville*,Stad Brussel*,,,1000102011201130,"176,545",32.632.6 km2 (12.6 sq mi),"5,415"
5,5,Etterbeek,Etterbeek,,,1040,"47,414",03.13.1 km2 (1.2 sq mi),"15,295"


In [37]:
df_final['Latitude'] = df_final.apply(lambda row: return_latlng(str(row.DutchName))['lat'], axis =1)
df_final['Longitude'] = df_final.apply(lambda row: return_latlng(str(row.DutchName))['lng'], axis =1)
df_final.head()

,SR,FrenchName,DutchName,Postalcode,Population,area,density,Latitude,Longitude
1,1,Anderlecht,Anderlecht,1070,"118,241",17.717.7 km2 (6.8 sq mi),"6,680",50.838141,4.312340
2,2,Auderghem,Oudergem,1160,"33,313",09.09.0 km2 (3.5 sq mi),"3,701",50.815657,4.433139
3,3,Berchem-Sainte-Agathe,Sint-Agatha-Berchem,1082,"24,701",02.92.9 km2 (1.1 sq mi),"8,518",50.863984,4.292702
4,4,Bruxelles-Ville*,Stad Brussel*,1000102011201130,"176,545",32.632.6 km2 (12.6 sq mi),"5,415",50.844041,4.367202
5,5,Etterbeek,Etterbeek,1040,"47,414",03.13.1 km2 (1.2 sq mi),"15,295",50.836851,4.389510


## Method to get venue details from foursquare APIs

In [38]:
CLIENT_ID = 'X1KOR4P2CDICF3QEA1XBSEWJSXME2FEI0A3SSTSRDXNZXNAO' # your Foursquare ID
CLIENT_SECRET = 'AOUOSTLGUU3B2VWPDWBB11ASJB4OHDZWCVVRH2XEYIZSUYLH' # your Foursquare Secret
VERSION = '20200707'

In [39]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
Brussels_venues = getNearbyVenues(names=df_final['DutchName'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )
Brussels_venues.head()

Anderlecht
Oudergem
Sint-Agatha-Berchem
Stad Brussel*
Etterbeek
Evere
Vorst
Ganshoren
Elsene
Jette
Koekelberg
Sint-Jans-Molenbeek
Sint-Gillis
Sint-Joost-ten-Node
Schaarbeek
Ukkel
Watermaal-Bosvoorde
Sint-Lambrechts-Woluwe
Sint-Pieters-Woluwe


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anderlecht,50.838141,4.31234,Friture René,50.835846,4.311632,Belgian Restaurant
1,Anderlecht,50.838141,4.31234,Le Chapeau Blanc,50.835034,4.307790,Restaurant
2,Anderlecht,50.838141,4.31234,Erasmushuis / Maison d'Erasme (Erasmushuis),50.836507,4.307860,History Museum
3,Anderlecht,50.838141,4.31234,Ulysse,50.838612,4.306860,Greek Restaurant
4,Anderlecht,50.838141,4.31234,Snack Mirvan,50.835176,4.308543,Snack Place


## Brussels map with all the municipalities marked

In [41]:
# create map of Brussels using latitude and longitude values
map_brussels = folium.Map(location=[50.8503, 4.3517], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_final['Latitude'], df_final['Longitude'], df_final['DutchName']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brussels)  
    
map_brussels

In [42]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
Brussels_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Anderlecht,20,20,20,20,20,20
Elsene,24,24,24,24,24,24
Etterbeek,41,41,41,41,41,41
Evere,29,29,29,29,29,29
Ganshoren,27,27,27,27,27,27
Jette,39,39,39,39,39,39
Koekelberg,17,17,17,17,17,17
Oudergem,34,34,34,34,34,34
Schaarbeek,41,41,41,41,41,41


In [44]:
PlaceType = Brussels_venues['Venue Category'].unique()

## Groups created to club all the retieved venue types these groups will be used for further clustering of municipalities.

In [45]:
CAT_utilities = ['Liquor Store','Shoe Repair','Lingerie Store','Gas Station','Mobile Phone Shop', 'Bike Rental / Bike Share','Boutique','Gourmet Shop','Cosmetics Shop','Beer Store','Bath House', 'Costume Shop','Health Food Store','Butcher','Toy / Game Store','Electronics Store','Market','Pharmacy',"Men's Store",'Factory','Church', 'Creperie', 'Pet Store', 'Bus Stop', 'Flower Shop', 'Paper / Office Supplies Store', 'Hobby Shop', 'Video Game Store','Fruit & Vegetable Store','Kids Store', 'Farmers Market','Grocery Store','Convenience Store','Discount Store','Plaza','Post Office','Bakery','Supermarket','Clothing Store','Furniture / Home Store']
CAT_Sports = ['Climbing Gym','Basketball Court','Moving Target','Trail','Skating Rink','Club House','Juice Bar','Tennis Court', 'Sports Bar','Hockey Field','Pool','Football Stadium','Sporting Goods Shop','Martial Arts Dojo','Volleyball Court','Soccer Field','Athletics & Sports', 'Dog Run','Gym','Gym / Fitness Center']
CAT_Econimical =['Wine Shop', 'Friterie', 'Deli / Bodega','Sports Club','Pub', 'Food & Drink Shop', 'Bagel Shop','Food Service','Halal Restaurant', 'Gastropub', 'Dessert Shop', 'Wine Bar','Cafeteria','Falafel Restaurant','Wings Joint','Fried Chicken Joint','Kebab Restaurant','Restaurant','Snack Place','Sandwich Place','Fast Food Restaurant','Ice Cream Shop','Salad Place','Burger Joint','Café','Beer Garden','Food Truck','Coffee Shop','Chocolate Shop','Breakfast Spot','Pizza Place']
CAT_luxry=['Swiss Restaurant','Spanish Restaurant','Event Service', 'Diner','Steakhouse','Lounge','Modern European Restaurant','Japanese Restaurant', 'Asian Restaurant', 'Brasserie','Eastern European Restaurant', 'Indian Restaurant','Chinese Restaurant', 'Bistro','Molecular Gastronomy Restaurant','Lebanese Restaurant', 'Theme Restaurant','Beer Bar', 'Music Venue','Spa','Tapas Restaurant','Piano Bar','Indonesian Restaurant', 'Theater','Hookah Bar', 'Turkish Restaurant','Moroccan Restaurant','Modern European Restaurant','Belgian Restaurant','Greek Restaurant','Middle Eastern Restaurant','Bar','Cocktail Bar','Italian Restaurant','French Restaurant','Sushi Restaurant','Nightclub','Indie Movie Theater','Thai Restaurant','Vietnamese Restaurant','Scandinavian Restaurant']
CAT_TouristsPlaces=['Castle','Museum','Garden','Exhibit','Art Museum','Bus Station','Tram Station', 'Train Station','Art Gallery','Platform','Windmill','History Museum','Park','Metro Station','Historic Site','Cultural Center','Concert Hall','Performing Arts Venue','Hotel','Palace']
CAT_Education=['School', 'Library','Hostel','Bookstore']

In [46]:
def getvenueclass(i):    
    if i in CAT_utilities:
        return 'CAT_utilities'
    if i in CAT_Sports:
        return 'CAT_Sports'
    if i in CAT_Econimical:
        return 'CAT_Econimical'
    if i in CAT_luxry:
        return 'CAT_luxry'
    if i in CAT_TouristsPlaces:
        return 'CAT_TouristsPlaces'
    if i in CAT_Education:
        return 'CAT_Education'
         
    return i

In [47]:
Brussels_venues = Brussels_venues.rename(columns={'Venue Category': 'VenueCategory'})

In [48]:
Brussels_venues['VenueClass'] = Brussels_venues.apply(lambda row: getvenueclass(row.VenueCategory), axis =1)
#df_final.apply(lambda row: return_latlng(str(row.DutchName))['lat'], axis =1)

In [22]:
Brussels_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,VenueCategory,VenueClass
0,Anderlecht,50.838141,4.312340,Friture René,50.835846,4.311632,Belgian Restaurant,CAT_luxry
1,Anderlecht,50.838141,4.312340,Le Chapeau Blanc,50.835034,4.307790,Restaurant,CAT_Econimical
2,Anderlecht,50.838141,4.312340,Erasmushuis / Maison d'Erasme (Erasmushuis),50.836507,4.307860,History Museum,CAT_TouristsPlaces
3,Anderlecht,50.838141,4.312340,Ulysse,50.838612,4.306860,Greek Restaurant,CAT_luxry
4,Anderlecht,50.838141,4.312340,Snack Mirvan,50.835176,4.308543,Snack Place,CAT_Econimical
5,Anderlecht,50.838141,4.312340,Bospark / Parc Forestier (Bospark),50.840087,4.310731,Park,CAT_TouristsPlaces
6,Anderlecht,50.838141,4.312340,Le Nil,50.836718,4.313628,Middle Eastern Restaurant,CAT_luxry
7,Anderlecht,50.838141,4.312340,Proxy Delhaize,50.835256,4.310246,Grocery Store,CAT_utilities
8,Anderlecht,50.838141,4.312340,Carrefour Express,50.835387,4.310213,Convenience Store,CAT_utilities
9,Anderlecht,50.838141,4.312340,Panos,50.835544,4.310776,Sandwich Place,CAT_Econimical


In [49]:
df_grp = Brussels_venues.groupby(['Neighborhood','VenueClass']).size()

In [24]:
df_grp.head(200)

Neighborhood            VenueClass              
Anderlecht              CAT_Econimical               4
                        CAT_TouristsPlaces           3
                        CAT_luxry                    5
                        CAT_utilities                8
Elsene                  CAT_Econimical               7
                        CAT_Sports                   1
                        CAT_TouristsPlaces           1
                        CAT_luxry                   14
                        CAT_utilities                1
Etterbeek               Bike Rental / Bike Share     1
                        CAT_Econimical               3
                        CAT_Education                1
                        CAT_Sports                   1
                        CAT_TouristsPlaces           7
                        CAT_luxry                   21
                        CAT_utilities                7
Evere                   CAT_Econimical               5
                

## One hot encoding of the dataframe

In [50]:
# one hot encoding
brussels_onehot = pd.get_dummies(Brussels_venues[['VenueClass']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brussels_onehot['Neighborhood'] = Brussels_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brussels_onehot.columns[-1]] + list(brussels_onehot.columns[:-1])
brussels_onehot = brussels_onehot[fixed_columns]

brussels_onehot.head()

,Neighborhood,CAT_Econimical,CAT_Education,CAT_Sports,CAT_TouristsPlaces,CAT_luxry,CAT_utilities
0,Anderlecht,0,0,0,0,1,0
1,Anderlecht,1,0,0,0,0,0
2,Anderlecht,0,0,0,1,0,0
3,Anderlecht,0,0,0,0,1,0
4,Anderlecht,1,0,0,0,0,0


In [51]:
brussels_grouped = brussels_onehot.groupby('Neighborhood').mean().reset_index()
brussels_grouped

,Neighborhood,CAT_Econimical,CAT_Education,CAT_Sports,CAT_TouristsPlaces,CAT_luxry,CAT_utilities
0,Anderlecht,0.200000,0.000000,0.000000,0.150000,0.250000,0.400000
1,Elsene,0.291667,0.000000,0.041667,0.041667,0.583333,0.041667
2,Etterbeek,0.073171,0.024390,0.024390,0.170732,0.512195,0.195122
3,Evere,0.172414,0.034483,0.206897,0.137931,0.206897,0.241379
4,Ganshoren,0.222222,0.000000,0.037037,0.000000,0.370370,0.370370
5,Jette,0.256410,0.000000,0.076923,0.205128,0.256410,0.205128
6,Koekelberg,0.176471,0.058824,0.352941,0.117647,0.235294,0.058824
7,Oudergem,0.176471,0.000000,0.029412,0.117647,0.441176,0.235294
8,Schaarbeek,0.195122,0.000000,0.048780,0.195122,0.170732,0.390244
9,Sint-Agatha-Berchem,0.357143,0.000000,0.071429,0.142857,0.214286,0.214286


In [52]:
num_top_venues = 5

for hood in brussels_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brussels_grouped[brussels_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anderlecht----
                venue  freq
0       CAT_utilities  0.40
1           CAT_luxry  0.25
2      CAT_Econimical  0.20
3  CAT_TouristsPlaces  0.15
4       CAT_Education  0.00


----Elsene----
                venue  freq
0           CAT_luxry  0.58
1      CAT_Econimical  0.29
2          CAT_Sports  0.04
3  CAT_TouristsPlaces  0.04
4       CAT_utilities  0.04


----Etterbeek----
                venue  freq
0           CAT_luxry  0.51
1       CAT_utilities  0.20
2  CAT_TouristsPlaces  0.17
3      CAT_Econimical  0.07
4       CAT_Education  0.02


----Evere----
                venue  freq
0       CAT_utilities  0.24
1          CAT_Sports  0.21
2           CAT_luxry  0.21
3      CAT_Econimical  0.17
4  CAT_TouristsPlaces  0.14


----Ganshoren----
            venue  freq
0       CAT_luxry  0.37
1   CAT_utilities  0.37
2  CAT_Econimical  0.22
3      CAT_Sports  0.04
4   CAT_Education  0.00


----Jette----
                venue  freq
0      CAT_Econimical  0.26
1           CAT_luxr

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brussels_grouped['Neighborhood']

for ind in np.arange(brussels_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brussels_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Anderlecht,CAT_utilities,CAT_luxry,CAT_Econimical
1,Elsene,CAT_luxry,CAT_Econimical,CAT_utilities
2,Etterbeek,CAT_luxry,CAT_utilities,CAT_TouristsPlaces
3,Evere,CAT_utilities,CAT_luxry,CAT_Sports
4,Ganshoren,CAT_utilities,CAT_luxry,CAT_Econimical


In [56]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Anderlecht,CAT_utilities,CAT_luxry,CAT_Econimical
1,Elsene,CAT_luxry,CAT_Econimical,CAT_utilities
2,Etterbeek,CAT_luxry,CAT_utilities,CAT_TouristsPlaces
3,Evere,CAT_utilities,CAT_luxry,CAT_Sports
4,Ganshoren,CAT_utilities,CAT_luxry,CAT_Econimical
5,Jette,CAT_luxry,CAT_Econimical,CAT_utilities
6,Koekelberg,CAT_Sports,CAT_luxry,CAT_Econimical
7,Oudergem,CAT_luxry,CAT_utilities,CAT_Econimical
8,Schaarbeek,CAT_utilities,CAT_TouristsPlaces,CAT_Econimical
9,Sint-Agatha-Berchem,CAT_Econimical,CAT_utilities,CAT_luxry


In [57]:
# set number of clusters
kclusters = 5

brussels_grouped_clustering = brussels_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brussels_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 2, 3, 1, 2, 1, 3, 1], dtype=int32)

In [58]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,3,Anderlecht,CAT_utilities,CAT_luxry,CAT_Econimical
1,1,Elsene,CAT_luxry,CAT_Econimical,CAT_utilities
2,1,Etterbeek,CAT_luxry,CAT_utilities,CAT_TouristsPlaces
3,2,Evere,CAT_utilities,CAT_luxry,CAT_Sports
4,3,Ganshoren,CAT_utilities,CAT_luxry,CAT_Econimical
5,1,Jette,CAT_luxry,CAT_Econimical,CAT_utilities
6,2,Koekelberg,CAT_Sports,CAT_luxry,CAT_Econimical
7,1,Oudergem,CAT_luxry,CAT_utilities,CAT_Econimical
8,3,Schaarbeek,CAT_utilities,CAT_TouristsPlaces,CAT_Econimical
9,1,Sint-Agatha-Berchem,CAT_Econimical,CAT_utilities,CAT_luxry


In [59]:
# add clustering labels
brussels_merged = df_final

brussels_merged = brussels_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='DutchName')

brussels_merged.head() # check the last columns!

,SR,FrenchName,DutchName,Postalcode,Population,area,density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,1,Anderlecht,Anderlecht,1070,"118,241",17.717.7 km2 (6.8 sq mi),"6,680",50.838141,4.312340,3,CAT_utilities,CAT_luxry,CAT_Econimical
2,2,Auderghem,Oudergem,1160,"33,313",09.09.0 km2 (3.5 sq mi),"3,701",50.815657,4.433139,1,CAT_luxry,CAT_utilities,CAT_Econimical
3,3,Berchem-Sainte-Agathe,Sint-Agatha-Berchem,1082,"24,701",02.92.9 km2 (1.1 sq mi),"8,518",50.863984,4.292702,1,CAT_Econimical,CAT_utilities,CAT_luxry
4,4,Bruxelles-Ville*,Stad Brussel*,1000102011201130,"176,545",32.632.6 km2 (12.6 sq mi),"5,415",50.844041,4.367202,1,CAT_Econimical,CAT_luxry,CAT_TouristsPlaces
5,5,Etterbeek,Etterbeek,1040,"47,414",03.13.1 km2 (1.2 sq mi),"15,295",50.836851,4.389510,1,CAT_luxry,CAT_utilities,CAT_TouristsPlaces


In [60]:
# create map
map_clusters = folium.Map(location=[50.8503, 4.3517], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brussels_merged['Latitude'], brussels_merged['Longitude'], brussels_merged['DutchName'], brussels_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [61]:
brussels_merged.loc[brussels_merged['Cluster Labels'] == 0, brussels_merged.columns[[1] + list(range(5, brussels_merged.shape[1]))]]

,FrenchName,area,density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
13,Saint-Gilles,02.52.5 km2 (1.0 sq mi),"20,188",51.226813,4.116662,0,CAT_utilities,CAT_TouristsPlaces,CAT_luxry


In [62]:
brussels_merged.loc[brussels_merged['Cluster Labels'] == 1, brussels_merged.columns[[1] + list(range(5, brussels_merged.shape[1]))]]

,FrenchName,area,density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,Auderghem,09.09.0 km2 (3.5 sq mi),"3,701",50.815657,4.433139,1,CAT_luxry,CAT_utilities,CAT_Econimical
3,Berchem-Sainte-Agathe,02.92.9 km2 (1.1 sq mi),"8,518",50.863984,4.292702,1,CAT_Econimical,CAT_utilities,CAT_luxry
4,Bruxelles-Ville*,32.632.6 km2 (12.6 sq mi),"5,415",50.844041,4.367202,1,CAT_Econimical,CAT_luxry,CAT_TouristsPlaces
5,Etterbeek,03.13.1 km2 (1.2 sq mi),"15,295",50.836851,4.389510,1,CAT_luxry,CAT_utilities,CAT_TouristsPlaces
9,Ixelles,06.36.3 km2 (2.4 sq mi),"13,690",50.822285,4.381571,1,CAT_luxry,CAT_Econimical,CAT_utilities
10,Jette,05.05.0 km2 (1.9 sq mi),"10,387",50.877763,4.326090,1,CAT_luxry,CAT_Econimical,CAT_utilities
14,Saint-Josse-ten-Noode,01.11.1 km2 (0.4 sq mi),"24,650",50.853074,4.372336,1,CAT_luxry,CAT_Econimical,CAT_utilities
17,Watermael-Boitsfort,12.912.9 km2 (5.0 sq mi),"1,928",50.799394,4.415818,1,CAT_luxry,CAT_Econimical,CAT_utilities


In [63]:
brussels_merged.loc[brussels_merged['Cluster Labels'] == 2, brussels_merged.columns[[1] + list(range(5, brussels_merged.shape[1]))]]

,FrenchName,area,density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
6,Evere,05.05.0 km2 (1.9 sq mi),"8,079",50.870452,4.402160,2,CAT_utilities,CAT_luxry,CAT_Sports
11,Koekelberg,01.21.2 km2 (0.5 sq mi),"18,008",50.862263,4.325708,2,CAT_Sports,CAT_luxry,CAT_Econimical
18,Woluwe-Saint-Lambert,07.27.2 km2 (2.8 sq mi),"7,669",50.846693,4.428484,2,CAT_luxry,CAT_Sports,CAT_utilities


In [64]:
brussels_merged.loc[brussels_merged['Cluster Labels'] == 3, brussels_merged.columns[[1] + list(range(5, brussels_merged.shape[1]))]]

,FrenchName,area,density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,Anderlecht,17.717.7 km2 (6.8 sq mi),"6,680",50.838141,4.312340,3,CAT_utilities,CAT_luxry,CAT_Econimical
7,Forest,06.26.2 km2 (2.4 sq mi),"8,991",50.809143,4.317751,3,CAT_utilities,CAT_luxry,CAT_Econimical
8,Ganshoren,02.52.5 km2 (1.0 sq mi),"9,838",50.871240,4.317510,3,CAT_utilities,CAT_luxry,CAT_Econimical
12,Molenbeek-Saint-Jean,05.95.9 km2 (2.3 sq mi),"16,378",50.854355,4.322778,3,CAT_utilities,CAT_luxry,CAT_TouristsPlaces
15,Schaerbeek,08.18.1 km2 (3.1 sq mi),"16,425",50.867604,4.373712,3,CAT_utilities,CAT_TouristsPlaces,CAT_Econimical
16,Uccle,22.922.9 km2 (8.8 sq mi),"3,594",50.801820,4.337235,3,CAT_utilities,CAT_luxry,CAT_Econimical


In [65]:
brussels_merged.loc[brussels_merged['Cluster Labels'] == 4, brussels_merged.columns[[1] + list(range(5, brussels_merged.shape[1]))]]

,FrenchName,area,density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
19,Woluwe-Saint-Pierre,08.98.9 km2 (3.4 sq mi),"4,631",50.829243,4.443297,4,CAT_TouristsPlaces,CAT_Sports,CAT_Econimical


<h2>Based on the above clusters ABC companey will have to focus on marketing of below plans segments for any given clusters </h2> <br>
<h3>Cluster 0 </h3><br>
   Primary Venue classes CAT_utilities,CAT_TouristsPlaces  - <br>
   suitable for Family plans and treveller plans<br>
<h3>Cluster 1 </h3><br>
   Primary Venue classes CAT_luxry, CAT_utilities  <br>
   suitable for Economical and high end Family plans<br>
<h3>Cluster 2 </h3><br>
   Primary Venue classes CAT_luxry,CAT_Sports  <br>
   suitable for high end plans and sports packages  <br>
<h3>Cluster 3 </h3><br>
   Primary Venue classes CAT_Econimical <br>
   suitable for economical Family and individual plans<br>
<h3>Cluster 4 </h3><br>
   Primary Venue classes CAT_TouristsPlaces,CAT_Sports <br>
   suitable for travellers plans and  sports packages  <br><br> 